<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%204%20-%20Models/C1_W4_Lab_2_resnet-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Implementing ResNet

In this lab, you will continue exploring Model subclassing by building a more complex architecture. 

[Residual Networks](https://arxiv.org/abs/1512.03385) make use of skip connections to make deep models easier to train. 
- There are branches as well as many repeating blocks of layers in this type of network. 
- You can define a model class to help organize this more complex code, and to make it easier to re-use your code when building the model.
- As before, you will inherit from the [Model class](https://keras.io/api/models/model/) so that you can make use of the other built-in methods that Keras provides.

## Imports

In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer

## Implement Model subclasses

As shown in the lectures, you will first implement the Identity Block which contains the skip connections (i.e. the `add()` operation below. This will also inherit the Model class and implement the `__init__()` and `call()` methods.

![image](https://www.researchgate.net/publication/331364877/figure/fig3/AS:741856270901252@1553883726825/Left-ResNet50-architecture-Blocks-with-dotted-line-represents-modules-that-might-be.png)

In [2]:
class Identity(tf.keras.Model):
    def __init__(self,filters,kernel_size):
        super(Identity, self).__init__(name='')
        
        self.conv1 = tf.keras.layers.Conv2D(filters,1,padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        
        self.conv2 = tf.keras.layers.Conv2D(filters,3,padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.conv3 = tf.keras.layers.Conv2D(filters,1,padding='same')
        self.bn3 = tf.keras.layers.BatchNormalization()
        
        self.act = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
    
    def call(self,input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.act(x)
        x = self.add([x, input_tensor])
        return x

From there, you can build the rest of the ResNet model. 
- You will call your `IdentityBlock` class two times below and that takes care of inserting those blocks of layers into this network.

In [17]:
from tensorflow.keras import backend as K
def multihead():
    value_1 = tf.keras.layers.Input(shape=(18, 512))
    query_1 = tf.keras.layers.Input(shape=(18, 512))
    key_1 = tf.keras.layers.Input(shape=(18, 512))
    
    key = tf.keras.layers.Dense(512,activation='relu')(key_1)
    value = tf.keras.layers.Dense(512,activation='relu')(value_1)
    query = tf.keras.layers.Dense(512,activation='relu')(query_1)
    
    value = tf.keras.layers.Reshape([64, 8, 64])(value)
    query = tf.keras.layers.Reshape([64, 8, 64])(query)
    key = tf.keras.layers.Reshape([64, 8, 64])(key)
    
    attention = tf.keras.layers.Lambda(lambda x: tf.linalg.matmul(x[0], x[1], transpose_a=True)/64**(0.5),
                                      output_shape=(64, 8, 8))([query, key])
    attention_soft = tf.keras.layers.Lambda(lambda x: K.softmax(x), output_shape=(64, 8, 8))(attention)
    
    output = tf.keras.layers.Lambda(lambda x: tf.linalg.matmul(x[0], x[1], transpose_b=True),
                                   output_shape=(64,8,64))([attention_soft, value])
    
    output = tf.keras.layers.Reshape([18, 512])(output)

    output = tf.keras.layers.Add()([output, query_1])
    output = tf.keras.layers.Dense(512, activation="relu")(output)
    
    return tf.keras.Model(inputs=[query_1, key_1, value_1], outputs=output)

In [25]:
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3))

        # Use the Identity blocks that you just defined
        self.id1a = Identity(64, 3)
        self.id1b = Identity(64, 3)
        self.attn = multihead()
        
        self.reshape = tf.keras.layers.Reshape([64, 512])
        self.max = tf.keras.layers.MaxPooling1D(4, name='pool3')

        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)

        # insert the identity blocks in the middle of the network
        #x = self.id1a(x)
        #x = self.id1b(x)
        x = self.reshape(x)
        x = self.attn([x, x, x])

        #x = self.global_pool(x)
        return self.classifier(x)

## Training the Model

As mentioned before, inheriting the Model class allows you to make use of the other APIs that Keras provides, such as:
- training
- serialization
- evaluation

You can instantiate a Resnet object and train it as usual like below:

**Note**: If you have issues with training in the Coursera lab environment, you can also run this in Colab using the "open in colab" badge link.

In [26]:
# utility function to normalize the images and return (image, label) pairs.
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

# create a ResNet instance with 10 output units for MNIST
resnet = ResNet(10)
# print(resnet.summary())
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# load and preprocess the dataset
dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)

# train the model.
resnet.fit(dataset, epochs=1)

      1/Unknown - 1s 1s/step

InvalidArgumentError:  Input to reshape is a tensor with 165888 values, but the requested shape has 1048576
	 [[node res_net_9/reshape_29/Reshape (defined at <ipython-input-25-43f8bf90db11>:29) ]] [Op:__inference_distributed_function_18833]

Errors may have originated from an input operation.
Input Source operations connected to node res_net_9/reshape_29/Reshape:
 res_net_9/max_pooling2d_9/MaxPool (defined at <ipython-input-25-43f8bf90db11>:24)

Function call stack:
distributed_function
